In [32]:
import numpy as np
from skimage.morphology import local_minima

In [10]:
def load_data(datafile):
    rows = []
    with open(datafile, 'r') as fp:
        for row in fp.readlines():
            rows.append([int(i) for i in row.strip()])
    return np.array(rows)

In [39]:
testdata = load_data("input.txt")

In [23]:
local_minima(testdata)

array([[False,  True, False, False, False, False, False, False, False,
         True],
       [False, False, False, False, False, False, False, False, False,
        False],
       [False, False,  True, False, False, False, False, False, False,
        False],
       [False, False, False, False, False, False, False, False, False,
        False],
       [False, False, False, False, False, False, False, False, False,
        False]])

In [40]:
mask = local_minima(testdata, connectivity=1, allow_borders=True)

In [41]:
np.sum(testdata[mask] + 1)

548